In [ ]:
%load_ext autoreload

import os, sys
import networkx as nx

sys.path.insert(0, "../")
sys.path.insert(0, "/home/j/joppich/.local/lib/python3.11/site-packages/")
#otherwise leidenalg is used in an outdated version ...

import leidenalg as la
print("la", la.version)

%autoreload 2
from mikg.kgraph import *
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

from collections import defaultdict, Counter

import random
random.seed(42)

import pickle

In [ ]:
kg = KGraph()
kg.load_kgraph("../data/initial_base_graph.out")

In [ ]:
kg.get_edge_between_type()

In [ ]:
exprDF = pd.read_csv("../seq_data/expression_ctzone_mean_df.tsv", sep="\t")

In [ ]:
exprDFs = {}

for group in exprDF.group.unique():
    exprDFs[group] = exprDF[exprDF.group == group].copy()

print([x for x in exprDFs])

In [ ]:
exprKGs = {}

gsa = GenesetAnnotator()
mNS = MeanNetworkScorer()

for group in exprDFs:
    
    print(group)
    
    gKG = KGraph()
    
    gKG.load_kgraph("../data/initial_base_graph.out")

    gKG.add_gene_expression(exprDFs[group])
    mNS.score_nodes_from_properties(gKG)

    mNS.score(gKG)

    gsa.annotate_genesets(gKG, settype="disease", targettype="gene")
    gsa.annotate_genesets(gKG, settype="geneset", targettype="gene")
    gsa.annotate_genesets(gKG, settype="ncRNA", targettype="gene")
    gsa.annotate_genesets(gKG, settype="drug", targettype=["gene", "disease"])
            
    exprKGs[group] = gKG

In [ ]:
print(len(exprKGs))
import pickle

with open("zone_exprkgs.pickle", 'wb') as f:
    pickle.dump(exprKGs, f)